In [ ]:
# DNN_A with 1025 input and 2050 output
# DNN_C with 2050 input and 2050 output
# Half of the data is used to train DNN_A and the other half is given to trained DNN_A to generate data for DNN_C
# DNN_C has it's own cost function(customized)

In [1]:
#import libraries.
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
# import sounddevice as sd
import tensorflow as tf
# from separation import bss_eval_sources 
from pystoi.stoi import stoi 

Using TensorFlow backend.


In [6]:
#import libraries.
import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from keras.models import load_model

In [3]:
#read necessary files, contarins mixed and clean features.
def data_reading(i):
    temp = open( "/home/majid/Codes/Thesis/TIMIT/Organized/concatenated/Mixed" +str(i) + ".p", "rb")
    ftr = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    temp = open( "/home/majid/Codes/Thesis/TIMIT/Organized/concatenated/clean" +str(i) + ".p", "rb")
    target = pickle.load( temp ).T
    temp.close()
    return ftr, target

In [4]:
ftr , target = data_reading(0)

In [5]:
target.shape

(116351, 2050)

In [10]:
# X = ftr[0:100000]
# y = target[0:100000]
# X_test = ftr[100000:]
# Y_test = target[100000:]


In [ ]:
X = ftr
y = target

In [11]:
h_A = [1025,1025,1025]
h_C = [4100, 4100, 4100]

In [12]:
ftr = []
target = []

In [7]:
def DNN_C_loss(y_true,y_pred):
    c = 0.2 # c is equal to lambda in the paper
    term1 = K.sum(((y_true - y_pred)**2))
    term2 = K.sum((y_true[0:1025] - y_pred[1025:2050])**2) + K.sum((y_true[1025:2050] - y_pred[0:1025])**2)
    loss = term1 - c * term2
    return loss

In [14]:
def DNN_A():
        # create model
        model = Sequential()
        model.add(Dense(h_A[0], input_dim = X.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
        model.add(Dense(y.shape[1], kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model

In [ ]:
def DNN_C():
        # create model
        model = Sequential()
        model.add(Dense(h_C[0], input_dim = X.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_C[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_C[2], kernel_initializer='normal', activation='relu'))
        model.add(Dense(y.shape[1], kernel_initializer='normal'))
        # Compile model
        model.compile(loss=DNN_C_loss, optimizer='adam')
        return model

In [17]:
# seed = 1
# np.random.seed(seed)
estimator_A = KerasRegressor(build_fn=DNN_A, epochs=1, batch_size=150, verbose=1)
# kfold = KFold(n_splits=2, random_state=seed)
kfold = KFold(n_splits=2)
results = cross_val_score(estimator_A, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
estimator_A.fit(X, y)
# prediction = estimator_A.predict(X_test)

Epoch 1/1
50000/50000 [==============================] - 5s 97us/step
Epoch 1/1
50000/50000 [==============================] - 5s 102us/step
Results: -0.15 (0.00) MSE
Epoch 1/1
16351/16351 [==============================] - 2s 107us/step


In [19]:
# Save a model you have trained
estimator_A.model.save('/home/majid/Codes/Thesis/Models/Two_stage/trained_model.h5')

In [23]:
# fit the model with the first half of dataset
for i in range(1,2):
    estimator_A = load_model('/home/majid/Codes/Thesis/Models/Two_stage/trained_model.h5')
    ftr , target = data_reading(i)
    X = ftr
    y = target
    ftr = []
    target = []
    estimator_A.fit(X, y, epochs=1, batch_size=150)
    estimator_A.model.save('/home/majid/Codes/Thesis/Models/Two_stage/trained_model.h5')

Epoch 1/1
119115/119115 [==============================] - 74s 618us/step - loss: 0.1277


/home/majid/anaconda3/envs/Myenv/lib/python3.6/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


Epoch 1/1
120729/120729 [==============================] - 75s 625us/step - loss: 0.1195
Epoch 1/1
116935/116935 [==============================] - 74s 632us/step - loss: 0.1171


In [8]:
# generating the second set of dataset
for i in range(2,4):
    estimator_A = load_model('/home/majid/Codes/Thesis/Models/Two_stage/trained_model.h5')
    ftr , target = data_reading(i)
    X_sec = ftr
#     y_sec = target
    ftr = []
    target = []
    prediction = estimator_A.predict(X_sec)
    pickle.dump( prediction, open( "/home/majid/Codes/Thesis/TIMIT/Organized/Two_stage_sec_set/new"+str(i)+".p", "wb" ))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [22]:
# from keras.callbacks import ModelCheckpoint
# # define the checkpoint
# filepath = "model.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [23]:
# # fit the model
# estimator_A.fit(X[300:600], y[300:600], epochs=10, batch_size=150, callbacks=callbacks_list)

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 1.6880

Epoch 00001: loss improved from inf to 1.68803, saving model to model.h5
Epoch 2/10
300/300 [==============================] - 0s 555us/step - loss: 0.2899

Epoch 00002: loss improved from 1.68803 to 0.28990, saving model to model.h5
Epoch 3/10
300/300 [==============================] - 0s 540us/step - loss: 0.2072

Epoch 00003: loss improved from 0.28990 to 0.20718, saving model to model.h5
Epoch 4/10
300/300 [==============================] - 0s 625us/step - loss: 0.1929

Epoch 00004: loss improved from 0.20718 to 0.19287, saving model to model.h5
Epoch 5/10
300/300 [==============================] - 0s 514us/step - loss: 0.1905

Epoch 00005: loss improved from 0.19287 to 0.19050, saving model to model.h5
Epoch 6/10
300/300 [==============================] - 0s 508us/step - loss: 0.1889

Epoch 00006: loss improved from 0.19050 to 0.18888, saving model to model.h5
Epoch 7/10
300/300 [====================

In [25]:
# fit the model
# estimator_A.fit(X[300:600], y[300:600])

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 1.6880
Epoch 2/10
300/300 [==============================] - 0s 542us/step - loss: 0.2993
Epoch 3/10
300/300 [==============================] - 0s 567us/step - loss: 0.2086
Epoch 4/10
300/300 [==============================] - 0s 570us/step - loss: 0.1932
Epoch 5/10
300/300 [==============================] - 0s 592us/step - loss: 0.1905
Epoch 6/10
300/300 [==============================] - 0s 577us/step - loss: 0.1889
Epoch 7/10
300/300 [==============================] - 0s 578us/step - loss: 0.1864
Epoch 8/10
300/300 [==============================] - 0s 586us/step - loss: 0.1829
Epoch 9/10
300/300 [==============================] - 0s 576us/step - loss: 0.1787
Epoch 10/10
300/300 [==============================] - 0s 639us/step - loss: 0.1741
